## Home Assitant 

I can manage different devices (door, light, etc.) from different zones (outdoor, kitchen, bedroom, etc.). The goal of the demo is to enable the user to perform several actions:
  
- List the available zones
- List the devices and their status in a zone
- Change the status of a device in a zone

database in JSON

```json
{
    1: {'zone': 'kitchen', 'devices': {'light': True, 'door': False}},
    2: {'zone': 'outdoor', 'devices': {'light': True, 'camera': True}},
}
```


In [49]:
import json
data = [{'zone': 'kitchen', 'devices': {'light': True, 'door': False}}, {'zone': 'outdoor', 'devices': {'light': True, 'camera': True}}]


class IntentsList:
    def list_available_zones(self) -> str:
        """ Return all available zones in the database. """
        return json.dumps(data)

    def list_device_status_by_zone(self,zone:str) -> str:
        for item in data:
            if item.get("zone") == zone:
                print("Device status in zone {}:".format(zone))
                return json.dumps(item['devices'])
        
        return "Zone not found."
    
    def update_zone_device_status(self,zone:str, device:str, status: bool):
        """ Not implemented yet.  Update the status of a device in a zone.  """
        for item in data:
            if item.get("zone") == zone:
                item['devices'][device] = status
                print("Update status of device {} in zone {} to {}".format(device, zone, status))
        return "Zone not found."
    
  

def list_device_status_by_zone_schema():
    return {
            "type": "function",
            "function": {
                "name": "list_device_status_by_zone",
                "description": "List the status of devices in a specific zone.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "zone": {"type": "string", "description": "The zone to list the device status for. Can be 'kitchen' or 'outdoor'."}
                    },
                    "required": ["zone"]
                }
            }
            
    }
    

def list_available_zones_schema():
    return {
        "type": "function",
        "function": {
            "name": "list_available_zones",
            "description": "List the available zones of the house.",
            "parameters": {"type": "object", "properties": {}},
            
        },
    } 

    

    
def update_zone_device_status_schema():
    return {
        "type": "function",
        "function": {
            "name": "update_zone_status",
            "description": "Update the status of a device in a specific zone.",
            "parameters": {
                "type": "object",
                "properties": {
                    "zone": {
                        "type": "string",
                        "description": "The zone to update the status for. Can be 'kitchen' or 'outdoor'.",
                    },
                    "device": {
                        "type": "string",
                        "description": "The device to update the status for. Can be 'light', 'door', or 'camera'.",
                    },
                },
                "required": ["zone", "device"],
            },
        },
    }
    


Run the assistant when a new user message is received
When a user sends a message, the following steps must be taken:

- Add the message to the list of messages (conversation)
- Execute the LLM by adding the available functions
- Add the LLM’s intermediate response to the message list
- Execute the pending functions (according to the LLM’s intermediate response)
- Add the functions’ responses to the list of messages
- Execute the LLM and return the response to the user

In [50]:
import os 
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv(override=True)

# print(os.getenv('OPENAI_API_KEY'))

client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))
tools = [list_available_zones_schema(), list_device_status_by_zone_schema(), update_zone_device_status_schema()]

content = 'What are the available zones?'
# content = "List device statuses in the kitchen zone?"
# content = "Update status of the light in the kitchen"

messages = [
        {"role": "system", "content": "You are a helpful assistant. Use functions if appropriate."},
        {"role": "user", "content": content},
]

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages,
    tools=tools,
    tool_choice='auto'
)

response


ChatCompletion(id='chatcmpl-9s54wWwqfss6JMu6lpKzL0HMIwzgQ', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_AVvmnJDua6q0lA9FYJoBqjLx', function=Function(arguments='{}', name='list_available_zones'), type='function')]))], created=1722676262, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier=None, system_fingerprint='fp_0f03d4f0ee', usage=CompletionUsage(completion_tokens=12, prompt_tokens=176, total_tokens=188))

In [51]:
response

ChatCompletion(id='chatcmpl-9s54wWwqfss6JMu6lpKzL0HMIwzgQ', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_AVvmnJDua6q0lA9FYJoBqjLx', function=Function(arguments='{}', name='list_available_zones'), type='function')]))], created=1722676262, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier=None, system_fingerprint='fp_0f03d4f0ee', usage=CompletionUsage(completion_tokens=12, prompt_tokens=176, total_tokens=188))

In [52]:
import json

intents_list_obj = IntentsList()


if response.choices[0].finish_reason == "tool_calls":
    # ! Hard code 1st tool
    function_name = response.choices[0].message.tool_calls[0].function.name
    args = response.choices[0].message.tool_calls[0].function.arguments
    tool_id = response.choices[0].message.tool_calls[0].id

    print("Function name:", function_name)
    print("Arguments:", args)

    if hasattr(intents_list_obj, function_name):
        function_to_call = getattr(intents_list_obj, function_name)
        args_dict = json.loads(args)
        output = function_to_call(**args_dict)
        
        messages.append(response.choices[0].message)
        messages.append({
            "role": "tool",
            "tool_call_id": tool_id,
            "content": output
        })

        # call llm again
        response = client.chat.completions.create(
            model="gpt-4o-mini", messages=messages
        )

        print("Final answer", response.choices[0].message.content)

Function name: list_available_zones
Arguments: {}
Final answer The available zones are:

1. **Kitchen**
   - Devices:
     - Light: Available
     - Door: Not Available

2. **Outdoor**
   - Devices:
     - Light: Available
     - Camera: Available
